# Imports

In [1]:
import datetime
import json
import os
import pickle
from itertools import chain, islice
from multiprocessing import Pool, cpu_count
from pathlib import Path
from pprint import pprint

import dateutil.parser
import findspark
import lmdb
import numpy as np
import pyspark
from ase.atoms import Atoms

from functools import partial
from ase.io.cfg import read_cfg
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    BooleanType,
    DoubleType,
    FloatType,
    IntegerType,
    LongType,
    StringType,
    StructField,
    StructType,
    TimestampType,
)

from colabfit.tools.configuration import AtomicConfiguration

from colabfit.tools.database import DataManager, PGDataLoader
from colabfit.tools.property import Property, property_object_schema
from colabfit.tools.configuration import config_schema
from colabfit.tools.property_definitions import (
    atomic_forces_pd,
    cauchy_stress_pd,
    potential_energy_pd,
)

findspark.init()
format = "jdbc"
load_dotenv("./.env")

True

# Set up MTPU and Carolina Materials readers and data

In [2]:
def convert_stress(keys, stress):
    stresses = {k: s for k, s in zip(keys, stress)}
    return [
        [stresses["xx"], stresses["xy"], stresses["xz"]],
        [stresses["xy"], stresses["yy"], stresses["yz"]],
        [stresses["xz"], stresses["yz"], stresses["zz"]],
    ]


SYMBOL_DICT = {"0": "Si", "1": "O"}


def reader(filepath):
    with open(filepath, "rt") as f:
        energy = None
        forces = None
        coords = []
        cell = []
        symbols = []
        config_count = 0
        for line in f:
            if line.strip().startswith("Size"):
                size = int(f.readline().strip())
            elif line.strip().lower().startswith("supercell"):
                cell.append([float(x) for x in f.readline().strip().split()])
                cell.append([float(x) for x in f.readline().strip().split()])
                cell.append([float(x) for x in f.readline().strip().split()])
            elif line.strip().startswith("Energy"):
                energy = float(f.readline().strip())
            elif line.strip().startswith("PlusStress"):
                stress_keys = line.strip().split()[-6:]
                stress = [float(x) for x in f.readline().strip().split()]
                stress = convert_stress(stress_keys, stress)
            elif line.strip().startswith("AtomData:"):
                keys = line.strip().split()[1:]
                if "fx" in keys:
                    forces = []
                for i in range(size):
                    li = {
                        key: val for key, val in zip(keys, f.readline().strip().split())
                    }
                    symbols.append(SYMBOL_DICT[li["type"]])
                    if "cartes_x" in keys:
                        coords.append(
                            [
                                float(c)
                                for c in [
                                    li["cartes_x"],
                                    li["cartes_y"],
                                    li["cartes_z"],
                                ]
                            ]
                        )
                    elif "direct_x" in keys:
                        coords.append(
                            [
                                float(c)
                                for c in [
                                    li["direct_x"],
                                    li["direct_y"],
                                    li["direct_z"],
                                ]
                            ]
                        )

                    if "fx" in keys:
                        forces.append(
                            [float(f) for f in [li["fx"], li["fy"], li["fz"]]]
                        )

            elif line.startswith("END_CFG"):
                if "cartes_x" in keys:
                    config = AtomicConfiguration(
                        positions=coords, symbols=symbols, cell=cell
                    )
                elif "direct_x" in keys:
                    config = AtomicConfiguration(
                        scaled_positions=coords, symbols=symbols, cell=cell
                    )
                config.info["energy"] = energy
                if forces:
                    config.info["forces"] = forces
                config.info["stress"] = stress

                if "Si" in symbols and "O" in symbols:
                    config.info["input"] = {
                        "kpoint-scheme": "Monkhorst-Pack",
                        "kpoints": "11x11x11",
                        "kinetic-energy-cutoff": {
                            "val": 1224,
                            "units": "eV",
                        },
                    }
                    config.info["_name"] = f"{filepath.stem}_SiO2_{config_count}"
                elif "Si" in symbols:
                    config.info["input"] = {
                        "kpoint-scheme": "Monkhorst-Pack",
                        "kpoints": "8x8x8",
                        "kinetic-energy-cutoff": {
                            "val": 884,
                            "units": "eV",
                        },
                    }
                    config.info["_name"] = f"{filepath.stem}_Si_{config_count}"
                elif "O" in symbols:
                    config.info["input"] = {
                        "kpoint-scheme": "Monkhorst-Pack",
                        "kpoints": "gamma-point",
                        "kinetic-energy-cutoff": {
                            "val": 1224,
                            "units": "eV",
                        },
                    }
                    config.info["_name"] = f"{filepath.stem}_O_{config_count}"
                config_count += 1
                yield config
                forces = None
                stress = []
                coords = []
                cell = []
                symbols = []
                energy = None

In [3]:
mtpu_configs = reader(Path("data/mtpu_2023/Unified_training_set.cfg"))
data = [x for x in mtpu_configs]
data[0].configuration_summary()

{'nsites': 4,
 'elements': ['Si'],
 'nelements': 1,
 'elements_ratios': [1.0],
 'chemical_formula_anonymous': 'A',
 'chemical_formula_reduced': 'Si',
 'chemical_formula_hill': 'Si4',
 'dimension_types': [0, 0, 0],
 'nperiodic_dimensions': 0}

In [4]:
SOFTWARE = "VASP"
METHODS = "DFT-PBE"
CM_PI_METADATA = {
    "software": {"value": SOFTWARE},
    "method": {"value": METHODS},
    "input": {"value": {"IBRION": 6, "NFREE": 4}},
}

CM_PROPERTY_MAP = {
    "formation-energy": [
        {
            "energy": {"field": "energy", "units": "eV"},
            "per-atom": {"value": False, "units": None},
        }
    ],
    "_metadata": CM_PI_METADATA,
}
CO_MD = {
    key: {"field": key}
    for key in [
        "_symmetry_space_group_name_H-M",
        "_symmetry_Int_Tables_number",
        "_chemical_formula_structural",
        "_chemical_formula_sum",
        "_cell_volume",
        "_cell_formula_units_Z",
        "symmetry_dict",
        "formula_pretty",
    ]
}


def load_row(txn, row):
    try:
        data = pickle.loads(txn.get(f"{row}".encode("ascii")))
        return data
    except TypeError:
        return False


def config_from_row(row: dict, row_num: int):
    coords = row.pop("cart_coords")
    a_num = row.pop("atomic_numbers")
    cell = [
        row.pop(x)
        for x in [
            "_cell_length_a",
            "_cell_length_b",
            "_cell_length_c",
            "_cell_angle_alpha",
            "_cell_angle_beta",
            "_cell_angle_gamma",
        ]
    ]
    config = Atoms(scaled_positions=coords, numbers=a_num, cell=cell)
    symmetry_dict = {str(key): val for key, val in row.pop("symmetry_dict").items()}
    for key in symmetry_dict:
        key = str(key)
    config.info = row
    config.info["symmetry_dict"] = symmetry_dict
    config.info["name"] = f"carolina_materials_{row_num}"
    return AtomicConfiguration.from_ase(config)


def carmat_reader(fp: Path):
    parent = fp.parent
    env = lmdb.open(str(parent))
    txn = env.begin()
    row_num = 0
    rows = []
    while row_num <= 10000:
        row = load_row(txn, row_num)
        if row is False:
            env.close()
            break
        rows.append(row)
        yield config_from_row(row, row_num)
        row_num += 1
    env.close()
    return False
    # return rows

In [5]:
reader = carmat_reader(Path("data/carolina_matdb/base/all/data.mdb"))

In [6]:
carmat_configs = list(carmat_reader(Path("data/carolina_matdb/base/all/data.mdb")))
PI_METADATA = {
    "software": {"value": "Quantum ESPRESSO"},
    "method": {"value": "DFT-PBE"},
    "input": {"field": "input"},
}

PROPERTY_MAP = {
    "potential-energy": [
        {
            "energy": {"field": "energy", "units": "eV"},
            "per-atom": {"value": False, "units": None},
            # "_metadata": PI_METADATA,
        }
    ],
    "atomic-forces": [
        {
            "forces": {"field": "forces", "units": "eV/angstrom"},
            # "_metadata": PI_METADATA,
        },
    ],
    "cauchy-stress": [
        {
            "stress": {"field": "stress", "units": "GPa"},
            "volume-normalized": {"value": True, "units": None},
        }
    ],
    "_metadata": PI_METADATA,
}

In [7]:
# carmat_aconfigs = [
#     AtomicConfiguration(names=f"{i}").from_ase(c) for i, c in enumerate(carmat_configs)
# ]
for c in carmat_configs:
    c.set_metadata(CO_MD)
len(carmat_configs)

10001

# Connect to DB and run loader

In [8]:
JARFILE = os.environ.get("CLASSPATH")
spark = (
    SparkSession.builder.appName("PostgreSQL Connection with PySpark")
    .config("spark.jars", JARFILE)
    .getOrCreate()
)
url = "jdbc:postgresql://localhost:5432/colabfit"
user = os.environ.get("PGS_USER")
password = os.environ.get("PGS_PASS")
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver",
}
loader = PGDataLoader(appname="colabfit", env="./.env")

24/04/16 15:54:35 WARN Utils: Your hostname, arktos resolves to a loopback address: 127.0.1.1; using 172.24.21.25 instead (on interface enp5s0)
24/04/16 15:54:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/16 15:54:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 15:54:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
with open("formation_energy.json", "r") as f:
    formation_energy_pd = json.load(f)
dm = DataManager(
    nprocs=4,
    configs=carmat_configs,
    prop_defs=[formation_energy_pd],
    prop_map=CM_PROPERTY_MAP,
)

In [10]:
# co_po = list(dm.gather_co_po_in_batches())
# co_po0 = co_po[0]
# len(co_po0)

In [11]:
# dm.load_data_to_pg_in_batches(loader)

In [12]:
# cos, pos = zip(*co_po0)

In [13]:
# pprint(cos[0])

In [14]:
# pprint(pos[0])

In [15]:
# dateutil.parser.parse(
#     datetime.datetime.now(tz=datetime.timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
# )

In [16]:
keys_to_hash_po = [
    "metadata",
    "software",
    "method",
    "chemical_formula_hill",
    "potential_energy",
    "potential_energy_unit",
    "potential_energy_per_atom",
    "potential_energy_reference",
    "potential_energy_reference_unit",
    "potential_energy_property_id",
    "atomic_forces",
    "atomic_forces_unit",
    "atomic_forces_property_id",
    "cauchy_stress",
    "cauchy_stress_unit",
    "cauchy_stress_volume_normalized",
    "cauchy_stress_property_id",
    "free_energy",
    "free_energy_unit",
    "free_energy_per_atom",
    "free_energy_reference",
    "free_energy_reference_unit",
    "free_energy_property_id",
    "band_gap",
    "band_gap_unit",
    "band_gap_property_id",
    "formation_energy",
    "formation_energy_unit",
    "formation_energy_per_atom",
    "formation_energy_reference",
    "formation_energy_reference_unit",
    "formation_energy_property_id",
    "adsorption_energy",
    "adsorption_energy_unit",
    "adsorption_energy_per_atom",
    "adsorption_energy_reference",
    "adsorption_energy_reference_unit",
    "adsorption_energy_property_id",
    "atomization_energy",
    "atomization_energy_unit",
    "atomization_energy_per_atom",
    "atomization_energy_reference",
    "atomization_energy_reference_unit",
    "atomization_energy_property_id",
]
keys_to_hash_co = [
    "metadata",
    "atomic_numbers",
    "nperiodic_dimensions",
    "cell",
    "pbc",
    "positions",
]

In [17]:
co = carmat_configs[0]

In [18]:
from hashlib import sha512

In [19]:
def _format_for_hash(v):
    # for now all AtomicConfiguration UIs are defined to be ndarrays
    if isinstance(v, np.ndarray):
        if v.dtype in [np.half, np.single, np.double, np.longdouble]:
            return np.round_(v.astype(np.float64), decimals=16)
        elif v.dtype in [np.int8, np.int16, np.int32, np.int64]:
            return v.astype(np.int64)
        else:
            return v
    elif isinstance(v, str):
        return v.encode("utf-8")
    else:
        return v


def _hash(row, identifying_fields_list):
    identifiers = [row[i] for i in identifying_fields_list]
    _hash = sha512()
    ordering = np.lexsort(
        (row["positions"][:, 2], row["positions"][:, 1], row["positions"][:, 0])
    )
    for k, v in zip(identifying_fields_list, identifiers):

        if k in ["positions", "atomic_numbers"]:
            _hash.update(bytes(_format_for_hash(v[ordering])))
        else:
            _hash.update(bytes(_format_for_hash(v)))
    return int(_hash.hexdigest(), 16)

In [20]:
co_hash = _hash(co.to_spark_row(), keys_to_hash_co)
# co["hash"] = co_hash
# co["id"] = f"CO_{co_hash}"
# # print(f"{co_json['hash']} --> {co['hash']}")

In [21]:
co_hash

# 3215353274399126448032015691019048641401660156433607261124207472627017875655228236316118273526758619024733802153640960980405996939699860646952638612055395

3215353274399126448032015691019048641401660156433607261124207472627017875655228236316118273526758619024733802153640960980405996939699860646952638612055395

In [22]:
hash(co_hash)
# 1117744068447613324

1117744068447613324

# outer

In [ ]:
"""
Can we make the configuration and the property instance/data object at the same time?
In this way, we would only have to pass through the data one time.

Workflow:
create database access object
create data reader as function? of the database access object
reader returns ase.Atoms-style objects (AtomicConfiguration)
DOs and PIs are now one object
These DOs point to a configuration
The configuration may already exist in the database, so we keep track of the hash added to the DO


"""

In [ ]:
cos = json.load(Path("sample_db/co_ds1.json").open("r"))

In [ ]:
with open(Path("sample_db/co_ds1.json"), "r") as f:
    co_json = spark.sparkContext.parallelize(json.load(f))

In [ ]:
co = co_json.map(_parse_config).map(stringify_lists)
co_df = spark.createDataFrame(co, config_schema)

In [ ]:
def parse_configs(co_path, spark):
    with open(co_path, "r") as f:
        co_json = spark.sparkContext.parallelize(json.load(f))
    co = co_json.map(_parse_config).map(stringify_lists)
    co_df = spark.createDataFrame(co, config_schema)
    return co_df

In [ ]:
parse_configs("sample_db/co_ds1.json", spark).show()

In [ ]:
table_name = "co"

mode = "append"
url = "jdbc:postgresql://localhost:5432/colabfit"
properties = {"user": user, "password": password, "driver": "org.postgresql.Driver"}
co_df.write.jdbc(url=url, table=table_name, mode=mode, properties=properties)

In [ ]:
# co_df.write.